In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
#!python
from numpy import *
from numpy.random import *
%matplotlib inline
import os
import time
import numpy as np
import pandas as pd
import sys
import cPickle as pickle
from   random import *
import scipy.stats
import matplotlib.pyplot as plt
sys.path.insert(1,'./code')
from visualize import plot_raw_data, get_population_data, plot_results
from EM import TransformToProb, RunEM
import pyparticleest.models.nlg as nlg
import pyparticleest.simulator as simulator
import pyparticleest.utils.kalman as kalman
from scipy.stats import bernoulli
import pyparticleest.interfaces as interfaces
from scipy.optimize import newton

//anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
mat = np.loadtxt('train_data.txt')
Y = mat[0,1:]
T = len(Y)

In [30]:
# Forward particles
N = 100
# Backward trajectories
M = 10
# 
P0 = 0.1
Q = 0.1
#trial
trial = 50
#
coefficient = scipy.special.binom(trial,Y)
#
l2pi = math.log(2 * math.pi)

In [5]:
print coefficient.shape

(3000,)


In [6]:
def resample(weights):
  n = len(weights)
  indices = []
  C = [0.] + [sum(weights[:i+1]) for i in range(n)]
  u0, j = random(), 0
  for u in [(u0+i)/n for i in range(n)]:
    while u > C[j]:
      j+=1
    indices.append(j-1)
  return indices

In [7]:
def measure(X, Y, trial, t, coefficient):
    # log-pdf of p(y_t|x_t)
    return np.log(coefficient[t]) + X[t,:] * Y[t] - trial * np.log(1+np.exp(X[t,:]))

In [8]:
def forward(X, a, S, t):
    #log-pdf of p(x_t|x_{t-1})
    return -0.5 * (l2pi + np.log(S) + ((X[t,:]-a).ravel() ** 2) / S)

In [9]:
def lognormpdf_scalar(err, S):
    """
    Calculate gaussian probability density of all elements in err, when
    err[i] ~ N(0,S) and each element in err is a scalar
    """
    return -0.5 * (l2pi + np.log(S) + (err.ravel() ** 2) / S)

In [54]:
##Initialization
t = 0
X = np.empty([T,N])
ancestor = np.empty([T,N])
w = np.empty([T,N])
W = np.empty([T,N])
X[t,:] = np.random.normal(0, P0, N)
w[t,:] = measure(X, Y, trial, t, coefficient)
W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
W[t,:] /= np.sum(W[t,:])
ancestor[t,:]= X[t,resample(W[t,:])]

In [76]:
-0.5 * (l2pi + np.log(0.1) + ((X[0,:]-0).ravel() ** 2) / 0.1)

array([ 0.16269176,  0.22435861,  0.22393753,  0.22758019,  0.19707331,
        0.18852767,  0.17538783,  0.07321101,  0.23015788,  0.15046212,
        0.05448229,  0.22486604,  0.22123211,  0.13398932,  0.09044584,
        0.22916716,  0.2275387 ,  0.21618851,  0.2322066 ,  0.22595921,
        0.21963117,  0.21706957,  0.23014866,  0.2228169 ,  0.19508001,
        0.23043161,  0.23232251,  0.19279387,  0.21936762,  0.19842946,
        0.22201859,  0.22207478,  0.15469775,  0.2302647 ,  0.16164756,
        0.16806459,  0.23225038,  0.21920073,  0.23223438,  0.20352528,
        0.20368849,  0.15655562,  0.20804928,  0.05916656,  0.12277894,
        0.1094633 ,  0.22919828,  0.22923248,  0.13005533,  0.21640808,
        0.18300311,  0.23199618,  0.20281916,  0.21786122,  0.17344899,
        0.2249256 ,  0.16652367,  0.22864463,  0.23208424,  0.22888333,
        0.20216581,  0.19977203,  0.21896996,  0.23090709,  0.13153648,
        0.19133426,  0.11616311,  0.10365266,  0.21968608,  0.16

In [56]:
ancestor[0,:]

array([-0.03089925, -0.08400084, -0.10673911, -0.10673911, -0.14026026,
       -0.14026026, -0.14026026, -0.14026026, -0.1684685 , -0.1684685 ,
       -0.1684685 , -0.1684685 , -0.1684685 , -0.1684685 , -0.1684685 ,
       -0.1684685 , -0.05686036, -0.05528914, -0.05096351, -0.12462444,
       -0.12462444, -0.12462444, -0.11339262, -0.11339262, -0.05128993,
       -0.07593251, -0.07571726, -0.06972049, -0.1861115 , -0.1861115 ,
       -0.1861115 , -0.1861115 , -0.1861115 , -0.1861115 , -0.1861115 ,
       -0.1861115 , -0.1861115 , -0.1861115 , -0.1861115 , -0.15677418,
       -0.15677418, -0.15677418, -0.15677418, -0.15677418, -0.15677418,
       -0.14303754, -0.14303754, -0.14303754, -0.14303754, -0.05647289,
       -0.09934879, -0.09934879, -0.10854033, -0.10854033, -0.02634648,
       -0.05033474, -0.11879817, -0.11879817, -0.05920374, -0.05863783,
       -0.09868309, -0.09868309, -0.11331234, -0.11331234, -0.12949363,
       -0.12949363, -0.12949363, -0.13726201, -0.13726201, -0.13

In [64]:
%%time
for tt in range(1000):
    t=tt+1
    a = ancestor[tt,:]
    mode = a + t*Q*(Y[t] - np.exp(a)/(1 + np.exp(a)) * trial)
    variance = 1.0 / (1.0/((t)*Q) + trial * np.exp(a)/(1+np.exp(a))**2)
    X[t,:] = np.random.normal(mode,variance, N)
    w[t,:] = measure(X, Y, trial, t, coefficient) + forward(X, a, Q, t) - lognormpdf_scalar(X[t,:]-mode, variance)
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    ancestor[t,:]= X[t,resample(W[t,:])]

CPU times: user 991 ms, sys: 16.2 ms, total: 1.01 s
Wall time: 1.02 s


In [82]:
ancestor[10,:]

array([-3.44942478, -2.5171643 , -3.39666129, -2.96038258, -2.96038258,
       -3.47221574, -3.37864358, -3.37864358, -3.02561331, -3.02561331,
       -2.80749651, -2.80749651, -3.37057782, -3.29978459, -3.29978459,
       -2.9614373 , -2.9614373 , -3.25385534, -3.25385534, -3.30925635,
       -3.30925635, -3.38920533, -3.38920533, -3.08486092, -3.08486092,
       -3.61444115, -2.66615197, -3.2620356 , -3.2620356 , -2.96469005,
       -2.96469005, -3.41489542, -3.41489542, -3.30941898, -3.30941898,
       -2.86839805, -2.86839805, -2.78304381, -3.65216944, -3.24101293,
       -3.24101293, -3.2960859 , -3.2960859 , -3.55660892, -3.63817019,
       -3.32111902, -3.32111902, -3.18529648, -3.18529648, -3.44054698,
       -3.10418007, -3.10418007, -3.10418007, -3.37051981, -3.37051981,
       -3.3513808 , -3.3513808 , -3.06201039, -3.06201039, -3.44746712,
       -3.04607166, -3.04607166, -3.0727793 , -3.0727793 , -3.11474498,
       -3.11474498, -3.11474498, -2.65184987, -2.96753583, -2.96

In [ ]:
## for t > 0
for tt in range(1):
    t=tt+1
    a = ancestor[t,:]
    mode = a + t*Q*(Y[t] - np.exp(a)/(1 + np.exp(a)) * (Y[t] - trial * (1+np.exp(a))))
    variance = 1.0 / (t*Q + Y[t]*np.exp(a)/(1+np.exp(a))**2 + trial * np.exp(a)*(1-np.exp(a))/(1+np.exp(a))**3)
    X[t,:] = np.random.normal(mode,variance)
    w[t,:] = measure(X, Y, trial, t, coefficient) + forward(X, a, Q, t) - lognormpdf_scalar(X[t,:]-mode, variance)
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    ancestor[t+1,:]= X[t,resample(W[t,:])]

In [ ]:
1.0 / (t*Q + Y[t]*np.exp(a)/(1+np.exp(a))**2 + trial * np.exp(a)*(1-np.exp(a))/(1+np.exp(a))**3)

In [ ]:
np.exp(a)*(1-np.exp(a))/(1+np.exp(a))**3

In [ ]:
ancestor[1,:]

In [ ]:
def h(x):
    return x - ancestor + t*Q *(y[t] - np.exp(x)/(1+np.exp(x)))/(1+np.exp(x))

def dh(x):
    return 1 - t * Q

In [ ]:
def k(x):
    return x**3 -1

In [ ]:
def dk(x):
    return 3*x**2

In [ ]:
x0=0.4
print newton(k, x0, dk)